In [9]:
import pandas as pd
import numpy as np

In [17]:



df_dict = pd.read_excel(
    '../Data/Input/online_retail_II.xlsx',
    sheet_name=['Year 2009-2010', 'Year 2010-2011']
)

df = pd.concat(df_dict.values(), ignore_index=True)
df


,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom
...,...,...,...,...,...,...,...,...
1067366,581587,22899,CHILDREN'S APRON DOLLY GIRL,6,2011-12-09 12:50:00,2.10,12680.0,France
1067367,581587,23254,CHILDRENS CUTLERY DOLLY GIRL,4,2011-12-09 12:50:00,4.15,12680.0,France
1067368,581587,23255,CHILDRENS CUTLERY CIRCUS PARADE,4,2011-12-09 12:50:00,4.15,12680.0,France
1067369,581587,22138,BAKING SET 9 PIECE RETROSPOT,3,2011-12-09 12:50:00,4.95,12680.0,France


In [18]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['InvoiceDate'].dt.year.value_counts()
df['InvoiceDate'] 

0         2009-12-01 07:45:00
1         2009-12-01 07:45:00
2         2009-12-01 07:45:00
3         2009-12-01 07:45:00
4         2009-12-01 07:45:00
                  ...        
1067366   2011-12-09 12:50:00
1067367   2011-12-09 12:50:00
1067368   2011-12-09 12:50:00
1067369   2011-12-09 12:50:00
1067370   2011-12-09 12:50:00
Name: InvoiceDate, Length: 1067371, dtype: datetime64[ns]

In [19]:
df = df[['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate', 'Price', 'Customer ID', 'Country']]

In [20]:
df['InvoiceDate'] = pd.to_datetime(df['InvoiceDate'])
df['Quantity'] = df[ 'Quantity'].astype('int32')
df['Price'] = df['Price']. astype('float32')

In [21]:
df.isna().sum()

Invoice             0
StockCode           0
Description      4382
Quantity            0
InvoiceDate         0
Price               0
Customer ID    243007
Country             0
dtype: int64

In [22]:
#Handling Missing values(NaNs)
df['is_loyal_customer'] = np.where(df['Customer ID'].isna(), 0, 1)

In [23]:
df['Category'] = np.where(
    df['Description'].str.contains('SET|PACK|BAG', na=False),
    'Household',
    'Grocery'
)


In [24]:
Invalid_Price_Count = (df['Price'] <= 0).sum()
Invalid_Price_Count

6207

In [25]:
#impute
df['UnitPrice'] = df.groupby('Category')['Price'] \
                    .transform(lambda x: x.fillna(x.median()))

df = df[df['UnitPrice'] > 0]

# df['UnitPrice'] = (
#     df.groupby('Category')['Price']
#       .transform(lambda x: x.fillna(x.median()))
# )

# df = df.loc[df['UnitPrice'] > 0]

# df['UnitPrice']


In [26]:
# df['is_return'] = np.where(df['Quantity'] < 0, 1,0)
# df['Quantity'] = df['Quantity'].abs()

df['is_return'] = (df['Quantity'] < 0).astype(int)
df['Quantity'] = df['Quantity'].abs()


Outlier price capping (winsorization)

In [27]:
upper_price = df['Price'].quantile(0.99)
df['Price'] = np.clip(df['Price'], 0, upper_price)

Advance Numpy Operation 

In [28]:
#Broadcasting (Tax & Discount)
Gst = 18

Category - based discount(lookup)

In [29]:
discount_map = {
        'Household' : 5,
        'Grocery' : 10
}
df['Base_Discount'] = df['Category'].map(discount_map)

Boolean Masking(premium product)

In [30]:
Premium_Masking  = df['Price'] > 10
df['Extra_Discount'] = np.where(Premium_Masking, 5, 0) 

Total Discount (Vectorization)

In [31]:
df['Total_Discount'] = df['Base_Discount'] + df['Extra_Discount']

Vectorized Arithmetic

In [32]:
df['Price_After_Discount'] = df['Price'] * (1 - df['Total_Discount'] / 100)
df['Final_Price'] = df['Price_After_Discount'] * (1 + Gst)
df['Final_Price']

0          118.844997
1          115.425000
2          115.425000
3           35.909998
4           21.375000
              ...    
1067366     35.909998
1067367     70.965002
1067368     70.965002
1067369     89.347497
1067370    290.700000
Name: Final_Price, Length: 1061164, dtype: float64

In [33]:
# df_error = df[(df['Price'] > 10) & (df['Extra_Discount'] == 0)]

# print("Rows where Extra Discount FAILED:")
# print(df_error[['Category', 'Price', 'Base_Discount', 'Extra_Discount']].head())
# df['Extra_Discount'].value_counts()


Reshaping & Multidimensional Arrays

In [34]:
df['date'] = df['InvoiceDate'].dt.date

daily_demand = (
        df.groupby(['date', 'Country'])['Quantity']
        .sum()
        .unstack(fill_value = 0)
)
df['date'].tail(20)

1067351    2011-12-09
1067352    2011-12-09
1067353    2011-12-09
1067354    2011-12-09
1067355    2011-12-09
1067356    2011-12-09
1067357    2011-12-09
1067358    2011-12-09
1067359    2011-12-09
1067360    2011-12-09
1067361    2011-12-09
1067362    2011-12-09
1067363    2011-12-09
1067364    2011-12-09
1067365    2011-12-09
1067366    2011-12-09
1067367    2011-12-09
1067368    2011-12-09
1067369    2011-12-09
1067370    2011-12-09
Name: date, dtype: object

Store & Category Performance KPIs

In [35]:
# Revenue calaculation (Vectorized)

df['Revenue'] = np.round(df['Final_Price'] * df['Quantity'], 2)
(df['Revenue'])

0          1426.14
1          1385.10
2          1385.10
3          1723.68
4           513.00
            ...   
1067366     215.46
1067367     283.86
1067368     283.86
1067369     268.04
1067370     290.70
Name: Revenue, Length: 1061164, dtype: float64

In [36]:
Store_Kpi = (
            df.groupby('Country')
            .agg(
                Total_Revenue = ('Revenue', 'sum'),
                Total_Units = ('Quantity', 'sum'), 
                Total_Price = ('Price', 'mean')
            )
            .reset_index()
)
Store_Kpi['Total_Revenue'] = Store_Kpi['Total_Revenue'].round(2)
Store_Kpi['Total_Price'] = Store_Kpi['Total_Price'].round(2)

In [37]:
Store_Kpi = (
    df.groupby('Country')
      .agg(
          Total_Revenue=('Revenue', 'sum'),
          Total_Units=('Quantity', 'sum'),
          Total_Price=('Price', 'mean')
      )
      .reset_index()
)



In [38]:
# scientific notation (total_revenue e+05)
pd.options.display.float_format = '{:,.2f}'.format


In [24]:
Store_Kpi

,Country,Total_Revenue,Total_Units,Total_Price
0,Australia,"2,953,220.92",104782,2.85
1,Austria,"379,888.39",11850,3.33
2,Bahrain,"57,846.04",1403,3.49
3,Belgium,"1,108,932.48",35482,3.60
4,Bermuda,"21,632.25",2798,2.49
5,Brazil,"24,220.66",545,2.73
6,Canada,"75,160.97",3657,2.30
7,Channel Islands,"797,634.21",22491,4.17
8,Cyprus,"406,901.73",11284,4.04
9,Czech Republic,"15,976.57",750,2.21


# A/B Testing

In [39]:
Control_Revenue = np.sum(df['Price'] * df['Quantity'])
Treated_Revenue = np.sum(df['Final_Price'] * df['Quantity'])

Lift = np.round((Treated_Revenue - Control_Revenue )/ Control_Revenue * 100 , 2)
Lift

1623.13

Extract_Data

In [40]:
df.to_csv ('../Data/Output/Retail_Clean_Fact_Sales.csv', index = False)
Store_Kpi.to_csv('../Data/Output/Retail_Store_Kpi.csv', index = False)
daily_demand.to_csv('../Data/Output/Retail_Daily_Demand.csv')

In [ ]:

“Using the Online Retail II dataset, I built a Tesco-scale analytics pipeline applying advanced NumPy operations like broadcasting, masking, reshaping, and vectorized computation. I handled missing values, noisy retail data, optimized preprocessing for performance, and produced forecasting-ready and BI-ready datasets supporting store operations, category performance, and digital retail optimization.”